In [1]:
# Install required libraries
!pip install transformers torch accelerate Pillow
!pip install opencv-python-headless

In [2]:
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration
import requests
from PIL import Image

# Set the device to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the LLaVA model and processor from Hugging Face
model_id = "llava-hf/llava-1.5-7b-hf"

model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16, # Use float16 for memory efficiency
    low_cpu_mem_usage=True,
).to(device)

processor = AutoProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

In [3]:
import cv2
from PIL import Image

def extract_frames(video_path, seconds_per_frame=1):
    """
    Extracts frames from a video file at a specified interval.

    Args:
        video_path (str): The path to the video file.
        seconds_per_frame (int): The interval in seconds to extract frames.

    Returns:
        list: A list of frames as PIL Image objects.
    """
    frames = []
    vidcap = cv2.VideoCapture(video_path)
    if not vidcap.isOpened():
        print(f"Error: Could not open video file at {video_path}")
        return frames

    fps = vidcap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(fps * seconds_per_frame)
    frame_count = 0

    success, image = vidcap.read()
    while success:
        if frame_count % frame_interval == 0:
            # Convert the frame from BGR (OpenCV format) to RGB (PIL format)
            rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            pil_image = Image.fromarray(rgb_image)
            frames.append(pil_image)
        success, image = vidcap.read()
        frame_count += 1

    vidcap.release()
    print(f"Extracted {len(frames)} frames from the video.")
    return frames

In [4]:
def generate_description_for_frame(image, model, processor, device):
    """
    Generates a descriptive text for a single image frame using LLaVA.

    Args:
        image (PIL.Image): The image frame to describe.
        model: The loaded LLaVA model.
        processor: The loaded LLaVA processor.
        device (str): The device to run inference on ('cuda' or 'cpu').

    Returns:
        str: The generated textual description.
    """
    # This prompt is tailored to your project's needs.
    # It specifically asks for actions, environment, and potential sounds.
    prompt = "USER: <image>\nDescribe this scene in detail, focusing on actions, the environment, and any objects that might produce sound. What sounds would you expect to hear?"

    inputs = processor(text=prompt, images=image, return_tensors="pt").to(device)

    # Generate the response
    generate_ids = model.generate(**inputs, max_new_tokens=200)

    # Decode the generated tokens into text, skipping special tokens
    response_text = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

    # Clean up the output to get only the assistant's response
    # The raw output includes the initial prompt, which we remove here.
    cleaned_response = response_text.split("ASSISTANT:")[-1].strip()

    return cleaned_response

In [5]:
# --- Main Execution ---

# 1. Define the path to your video file
# Make sure you have uploaded the video to your Colab session
video_path = "your_video_file.mp4" # <--- IMPORTANT: Change this to your file's name

# 2. Extract frames from the video
print(f"Processing video: {video_path}")
game_frames = extract_frames(video_path)

# 3. Generate a description for each frame
all_descriptions = []
if game_frames:
    for i, frame in enumerate(game_frames):
        print(f"Generating description for frame {i+1}/{len(game_frames)}...")
        description = generate_description_for_frame(frame, model, processor, device)
        all_descriptions.append(description)
        print(f"  -> Description: {description}\n")

# 4. Display all generated descriptions
print("\n--- All Generated Descriptions ---")
for i, desc in enumerate(all_descriptions):
    print(f"Frame {i+1}: {desc}")

# The 'all_descriptions' list now contains the prompts for AudioLDM2.

Processing video: your_video_file.mp4
Error: Could not open video file at your_video_file.mp4

--- All Generated Descriptions ---
